In [1]:
import json

In [2]:
tweets = {}
hashtag_dict = {}

In [3]:
with open('../data/deletweet_subset_10000.json', 'r') as f:
    
    for line in f:
        tweet = json.loads(line)
        tweets[tweet['id']] = tweet

In [4]:
len(tweets)

10000

In [5]:
for key in tweets.keys():
    tweet = tweets[key]
    if (tweet['entities']['hashtags']):
        hashtag_dict[key] = [tweet['entities']['hashtags'][i]['text'] for i in range(len(tweet['entities']['hashtags']))]

In [6]:
# how many tweets in the sample have hashtags
len(hashtag_dict)

4830

In [7]:
tags = []
for key in hashtag_dict.keys():
    for tag in hashtag_dict[key]:
        tags.append(tag)

# how many individual hashtags are in the sample
len(tags)

7821

In [8]:
# how many unique hashtags are in the sample
len(set(tags))

4300